## Getting Started

In [ ]:
!pip install sqlalchemy-bigquery langchain-experimental==0.0.17 langchain==0.0.240 pydantic==1.10.12 gradio==3.44.4

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
PROJECT_ID = "PROJECT_ID"
GOOGLE_APPLICATION_CREDENTIALS = "../credentials.json"
SERVICE_ACCOUNT = "SERVICE_ACCOUNT"

In [ ]:
!gcloud config set account {SERVICE_ACCOUNT}
!gcloud auth activate-service-account --key-file={GOOGLE_APPLICATION_CREDENTIALS}
!gcloud config set project {PROJECT_ID}

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID)

In [ ]:
from langchain.llms import VertexAI

In [ ]:
llm = VertexAI(model_name='text-bison@001',
               temperature=0, max_output_tokens=1024)

In [ ]:
llm("tell me a story about mice")

## Setup DB connection

In [ ]:
from langchain import SQLDatabase

In [ ]:
langchain_db = SQLDatabase.from_uri(f"bigquery://{PROJECT_ID}/hackathon")

In [ ]:
langchain_db.run("SELECT count(*) FROM orders WHERE order_date BETWEEN '2023-01-01' AND '2023-01-31'")

## Create Langchain SQL Chain

In [ ]:
from langchain.chains import SQLDatabaseSequentialChain

In [ ]:
db_chain = SQLDatabaseSequentialChain.from_llm(
        llm,
        langchain_db,
        verbose=True,
        return_intermediate_steps=True,
)

In [ ]:
output = db_chain("how many items did I sell in January 2023")
output["result"]

In [ ]:
output["intermediate_steps"][1]

## Improving The Chain

In [ ]:
from langchain import PromptTemplate
from datetime import datetime

In [ ]:
CUSTOM_SQL_PROMPT = """
You are a GoogleSQL expert. Given an input question, first create a syntactically
correct GoogleSQL query to run, then look at the results of the query and return
the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most {top_k} results using the LIMIT clause as per GoogleSQL. You can
order the results to return the most informative data in the database.

Never query for all columns from a table. You must query only the columns that are
needed to answer the question. Wrap each column name and value in backticks (`)
to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below. Be careful
to not query for columns that do not exist. Also, pay attention to which column
is in which table.

Name all columns in the returned data appropriately. If a column does not have a
matching name in the schema, create an appropriate name reflecting its content.

Use the following format:

Question: "Question here"

SQLQuery: "SQL Query to run"

SQLResult: "Result of the SQLQuery"

Answer: "Final answer here"


Today''s date is {today_date}. When querying between dates, add the dates in quotes
('')

If someone asks for a specific month, use the range between the current month''s
start date and the current month''s end date.

If someone asks for a specific year, use the range between the first month of the
current year and the current month''s end date.


Remember to always use natural language when writing your final answer.

Only use the following tables:

{table_info}

Question: {question}
"""

In [ ]:
test_prompt = PromptTemplate(template=CUSTOM_SQL_PROMPT, input_variables=["question", "table_info", "today_date", "top_k"])

In [ ]:
output = db_chain(test_prompt.format(
    question = "how many items did I sell in January 2023",
    table_info = ["customers","employees","financial_goals","inventory","orders","product_reviews","supplier_orders"],
    today_date = datetime.now().strftime("%m/%d/%Y"),
    top_k=10
))
output["result"]

In [ ]:
from datetime import datetime

table_names = ["customers","employees","financial_goals","inventory","orders","product_reviews","supplier_orders"]

def create_sql_chain(question: str, table_info: str = table_names, top_k:int=100, llm: VertexAI = llm, db=langchain_db):
    """ Create a Q&A conversation chain using the VertexAI LLM.

    """
    
    db_chain = SQLDatabaseSequentialChain.from_llm(
        llm,
        db,
        verbose=True,
        return_intermediate_steps=True,
    )
    test_prompt = PromptTemplate(template=CUSTOM_SQL_PROMPT, input_variables=["question", "table_info", "today_date", "top_k"])

    today_date = datetime.now().strftime("%m/%d/%Y")
    output = db_chain(test_prompt.format(
        question=question,
        table_info=table_info,
        today_date=today_date,
        top_k=top_k
        ))
    sql_query = output["intermediate_steps"][1]
    response = output["result"]
    
    return response, sql_query

In [ ]:
create_sql_chain('how many items did I sell in January?')

## Chaining Chains

In [ ]:
from langchain.chains import LLMChain
from IPython.display import display, Markdown

In [ ]:
EMAIL_PROMPT = """You are an Analytics Assistant. Your task is to assist users to better understand the insights from their BigQuery datasets.
Users will ask a question to BigQuery and you will receive the question and the answer. 

Your task is to draft an email summarising the answer provided to the user.

The user question was:
```
{question}
```
and the answer provided was 
```
{answer}
```

Based on this, draft an email. Structure the email as follows:
1. Start with a cordial introduction
2. Remind the recipient as to what the user question was
3. Provide a summary of the answer
4. Send your best regards and say that you are happy to have a follow up

Answer here:

"""

email_prompt = PromptTemplate(template=EMAIL_PROMPT, input_variables=["question", "answer"])

In [ ]:
email_chain = LLMChain(
        llm=llm, prompt=email_prompt, output_key="output")

In [ ]:
display(Markdown(email_chain.run(
    {
        'question':"what is the meaning of life?", 
        'answer': "42"
    }
)))

In [ ]:
def combined_chain(
    question: str, 
    table_info: str = table_names, 
    top_k:int=100, 
    llm: VertexAI = llm, 
    db=langchain_db
):
    response, sql_query = create_sql_chain(
        question,
        table_info,
        top_k,
        llm,
        db
    )
    
    output = email_chain.run(
        {
            'question':question,
            'answer':response
        }
    )
    
    return output
    

In [ ]:
display(Markdown(combined_chain("how many items did I sell in January 2023")))

## Grad IO email interface:

In [ ]:
import gradio as gr

In [ ]:
# Create a Gradio interface
iface = gr.Interface(
    fn= combined_chain,  # Function to execute when a query is received
    inputs="text",      # Input is a single text field
    outputs="text",     # Output will be a text response
    title="Analytics Worker Demo",
    description="Enter a question, and the system will query the database and provide an answer.",
)

# Launch the Gradio interface on a specified port (e.g., 5000)
iface.launch(share=True)